In [ ]:
"""
File: XGBoost.ipynb
Code to train and evaluate an XGBoost model on MIMIC-IV FHIR dataset.
"""


def ProjectObjectives():
    """
    __Objectives__
    0. Import data and separate unique visit tokens
    1. Reduce the number of features (manual selection, hierarchy aggregation)
    2. Create frequency features from event tokens
    3. Include num_visits, youngest and oldest age, and maybe time
    4. Use label column to create the prediction objective
    5. Train XGBoost model and evaluate on test dataset
    >>> All objectives successful
    """
    return ProjectObjectives.__doc__

In [ ]:
# Import dependencies and define useful constants
import os
import pickle
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy.sparse import (
    csr_matrix,
    hstack,
    load_npz,
    save_npz,
    vstack,
)
from sklearn.metrics import (
    auc,
    average_precision_score,
    balanced_accuracy_score,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm


plt.style.use("seaborn-v0_8")
%matplotlib inline

ROOT = "/fs01/home/afallah/odyssey/odyssey"
os.chdir(ROOT)

TASK = "multi_v2"
NUM_PATIENTS = "all"
TASK_DURATION = "readmission_1month"
DATA_ROOT = f"{ROOT}/odyssey/data/bigbird_data"
DATA_PATH = f"{DATA_ROOT}/patient_sequences/patient_sequences_2048_{TASK}.parquet"
ID_PATH = f"{DATA_ROOT}/patient_id_dict/dataset_2048_{TASK}.pkl"
FREQ_MATRIX_TRAIN = f"{DATA_ROOT}/patient_freq_matrix/{TASK}/patient_freq_matrix_finetune_{TASK_DURATION}.npz"
FREQ_MATRIX_TEST = f"{DATA_ROOT}/patient_freq_matrix/{TASK}/patient_freq_matrix_test_{TASK_DURATION}.npz"

In [ ]:
# Load dataset
dataset = pd.read_parquet(DATA_PATH)
# dataset.rename(columns={"label_length_of_stay_1week": "label"}, inplace=True)
# dataset["label"] = dataset["label"].astype(int)
# dataset.rename(columns={f'all_conditions': 'label'}, inplace=True)
dataset.rename(columns={f"label_{TASK_DURATION}": "label"}, inplace=True)
dataset

In [ ]:
# Function to truncate event tokens based on cutoff
def truncate_tokens(row):
    return row["event_tokens_2048"][: row[f"cutoff_{TASK_DURATION}"]]


dataset = dataset[dataset[f"cutoff_{TASK_DURATION}"] != -1]
dataset["event_tokens_2048"] = dataset.apply(truncate_tokens, axis=1)
dataset.reset_index(drop=True, inplace=True)

In [ ]:
patient_ids = pickle.load(open(ID_PATH, "rb"))
train_data = dataset.loc[
    dataset["patient_id"].isin(patient_ids["finetune"]["few_shot"][NUM_PATIENTS])
]
test_data = dataset.loc[dataset["patient_id"].isin(patient_ids["test"])]

In [ ]:
# Find the unique set of all possible tokens, including special tokens
unique_event_tokens = set()

for patient_event_tokens in tqdm(
    dataset["event_tokens_2048"].values,
    desc="Loading Tokens",
    unit=" Patients",
):
    for event_token in patient_event_tokens:  # patient_event_tokens.split(" "):
        unique_event_tokens.add(event_token)

unique_event_tokens = list(unique_event_tokens)
unique_event_tokens.sort(reverse=True)

print(
    f"Complete list of unique event tokens\nLength: {len(unique_event_tokens)}\nHead: {unique_event_tokens[:30]}...",
)

In [ ]:
# Define the list of tokens being used as features for the XGBoost
# Note that feature 'id' will be dropped later on
special_tokens = [
    "[CLS]",
    "[PAD]",
    "[REG]",
    "[UNK]",
    "[VS]",
    "[VE]",
    "[W_0]",
    "[W_1]",
    "[W_2]",
    "[W_3]",
    *[f"[M_{i}]" for i in range(0, 13)],
    "[LT]",
]

feature_event_tokens = ["id"] + [
    token for token in unique_event_tokens if token not in special_tokens
]

print(len(feature_event_tokens), feature_event_tokens[:20])

In [ ]:
# Generate feature frequency matrix for each feature token in the patient data
# Since this will be saved to disk, you need only do it once


def get_patient_frequency_matrix(
    data: pd.DataFrame,
    feature_event_tokens: List[str],
    special_tokens: List[str],
    output_path: str,
    buffer_size: int = 50000,
) -> None:
    """Calculate and save the patient frequency matrix."""
    patient_freq_matrix = None
    matrix_buffer = []

    for idx, patient in tqdm(
        data.iterrows(),
        desc="Loading Tokens",
        unit=" Patients",
        total=len(data),
    ):
        patient_history = {token: 0 for token in feature_event_tokens}
        patient_history["id"] = idx

        for event_token in patient["event_tokens_2048"]:  # .split(" "):
            if event_token not in special_tokens:
                patient_history[event_token] += 1

        matrix_buffer.append(list(patient_history.values()))

        if len(matrix_buffer) >= buffer_size:
            current_matrix = csr_matrix(
                matrix_buffer,
                shape=(len(matrix_buffer), len(feature_event_tokens)),
            )

            if patient_freq_matrix is None:
                patient_freq_matrix = current_matrix
            else:
                patient_freq_matrix = vstack(
                    [patient_freq_matrix, current_matrix],
                    format="csr",
                )

            matrix_buffer = []

    if matrix_buffer:
        current_matrix = csr_matrix(
            matrix_buffer,
            shape=(len(matrix_buffer), len(feature_event_tokens)),
        )

        if patient_freq_matrix is None:
            patient_freq_matrix = current_matrix
        else:
            patient_freq_matrix = vstack(
                [patient_freq_matrix, current_matrix],
                format="csr",
            )

    save_npz(output_path, patient_freq_matrix)
    print(f"Save & Done! Final Matrix Shape: {patient_freq_matrix.shape}\n")


get_patient_frequency_matrix(
    train_data, feature_event_tokens, special_tokens, output_path=FREQ_MATRIX_TRAIN
)
get_patient_frequency_matrix(
    test_data, feature_event_tokens, special_tokens, output_path=FREQ_MATRIX_TEST
)

In [ ]:
# Load feature frequency matrix from disk, generated by above code cell
patient_freq_matrix = load_npz(FREQ_MATRIX_TRAIN)
patient_freq_matrix_test = load_npz(FREQ_MATRIX_TEST)
num_patients = patient_freq_matrix.shape[0] + patient_freq_matrix_test.shape[0]
patient_freq_matrix

In [ ]:
def find_min_greater_than_zero(lst: List[int]) -> int:
    """Return the minimum positive number in the given list"""
    positive_numbers = np.array(lst)[np.array(lst) > 0]

    if len(positive_numbers) == 0:
        return 0

    min_positive = np.min(positive_numbers)
    return min_positive

In [ ]:
# Get extra features such as number of visits and age
def add_age_to_freq_matrix(
    data: pd.DataFrame,
    patient_freq_matrix: np.ndarray,
) -> np.ndarray:
    """Add age feature to patient frequency matrix."""
    num_visits = data["num_visits"].values
    min_age = [
        find_min_greater_than_zero(patient_age_tokens)
        for patient_age_tokens in data["age_tokens_2048"]
    ]
    max_age = [
        np.max(patient_age_tokens) for patient_age_tokens in data["age_tokens_2048"]
    ]

    # Add extra features to the frequency dataset
    patient_freq_matrix = hstack(
        [patient_freq_matrix, csr_matrix([num_visits, min_age, max_age]).T],
        format="csr",
    )
    return patient_freq_matrix[:, 1:]  # Drop id feature


patient_freq_matrix = add_age_to_freq_matrix(train_data, patient_freq_matrix)
patient_freq_matrix_test = add_age_to_freq_matrix(test_data, patient_freq_matrix_test)

In [ ]:
# Get intuition about the frequency of different features in the training dataset
report_threshold = 1
features_above_threshold = np.sum(
    (patient_freq_matrix.getnnz(axis=0) >= report_threshold).astype(int),
)

print(
    f"How many features have been reported for at least {report_threshold} patients?\n"
    f"{features_above_threshold} Features",
)

# Plot the histogram of feature frequency
# plt.hist(patient_freq_matrix.getnnz(axis=0), bins=range(num_patients+1), edgecolor='black')
# plt.xlabel('Number of Nonzero Rows')
# plt.ylabel('Number of Columns')
# plt.title('Histogram of Nonzero Rows per Column')
# plt.show()

In [ ]:
# Pick features to train the model on
NUM_FEATURES = 20000
features_sorted_by_freq = np.argsort(-patient_freq_matrix.getnnz(axis=0))
selected_features = features_sorted_by_freq[: NUM_FEATURES + 1]
selected_features

In [ ]:
# Define custom labels, here death in 12 M, not needed if dataset already has labels
# data["label"] = (
#     (data["death_after_end"] >= 0) & (data["death_after_end"] <= 31)
# ).astype(int)
#
# print(f"Total positive labels: {sum(data['label'])} out of {len(data)}")

In [ ]:
# Prepare data for model training
X = vstack([patient_freq_matrix, patient_freq_matrix_test])
Y = hstack(
    [np.array(train_data["label"].tolist()), np.array(test_data["label"].tolist())]
)

# Optional, Scale features. Didn't improve performance
# scaler = MaxAbsScaler()
# X = scaler.fit_transform(X)

# Optional, Split data into train and test sets if dataset is not already split
# X_train, X_test, y_train, y_test = train_test_split(
#     X, Y, test_size=0.2, stratify=Y, random_state=1
# )

In [ ]:
# Split data into train and test based on original dataset definitions
X_train = patient_freq_matrix
X_test = patient_freq_matrix_test

y_train = np.array(train_data["label"].tolist())
y_test = np.array(test_data["label"].tolist())

patient_indices_train = y_train != -1
patient_indices_test = y_test != -1

X_train = X_train[patient_indices_train]
X_test = X_test[patient_indices_test]
y_train = y_train[patient_indices_train]
y_test = y_test[patient_indices_test]

In [ ]:
models = {"unweighted": [], "weighted1": [], "weighted2": [], "weighted3": []}
weights = {
    "unweighted": np.ones(shape=20),
    "weighted1": np.array(
        [
            5.69327441,
            7.42882028,
            10.13231198,
            11.49395911,
            16.40469558,
            11.0384684,
            15.53901244,
            10.03937008,
            12.32068141,
            12.45844666,
            50.0,
            50.0,
            50.0,
            50.0,
            50.0,
            50.0,
            50.0,
            50.0,
            50.0,
            50.0,
        ],
    ).tolist(),
    "weighted2": np.array(
        [
            1.0,
            1.30484142,
            1.77969851,
            2.01886617,
            2.88141663,
            1.93886112,
            2.72936299,
            1.76337365,
            2.16407651,
            2.1882744,
            339.421875,
            60.34166667,
            164.56818182,
            156.28057554,
            30.50983146,
            145.79194631,
            66.84,
            88.66530612,
            35.55319149,
            35.32195122,
        ],
    ).tolist(),
    "weighted3": np.array(
        [
            1.0,
            1.14229656,
            1.33405341,
            1.4208681,
            1.6974736,
            1.39242993,
            1.65207838,
            1.3279208,
            1.47108005,
            1.47928172,
            18.42340563,
            7.76798987,
            12.82841307,
            12.50122296,
            5.52357054,
            12.07443358,
            8.17557337,
            9.41622568,
            5.9626497,
            5.94322734,
        ],
    ).tolist(),
}

# Train a separate model for each label
for weighting_scheme in models:
    for i in range(y_train.shape[1]):
        model = xgb.XGBClassifier(
            objective="binary:logistic",
            random_state=23,
            scale_pos_weight=weights[weighting_scheme][i],
        )
        model.fit(X_train, y_train[:, i])
        models[weighting_scheme].append(model)
        print(f"Model saved for label: {i}")

In [ ]:
def generate_predictions(model, X):
    """
    Generate both class predictions and probabilities.
    """
    pred = model.predict(X)
    prob = model.predict_proba(X)[:, 1]
    return pred, prob


def calculate_metrics(y_true, y_pred, y_prob):
    """
    Calculate and return performance metrics.
    """
    metrics = {
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "F1 Score": f1_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "AUROC": roc_auc_score(y_true, y_prob),
        "Average Precision Score": average_precision_score(y_true, y_prob),
    }

    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    metrics["AUC-PR"] = auc(recall, precision)

    return metrics


def plot_roc_curve(y_true, y_prob, label_prefix):
    """
    Plot ROC curve for a dataset.
    """
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    plt.plot(
        fpr,
        tpr,
        label=f"{label_prefix} AUROC={roc_auc_score(y_true, y_prob):.3f}",
    )


def assess_model_performance(
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    X_all,
    Y_all,
    plot=False,
    print_metrics=False,
    return_test_metrics=True,
):
    """
    Assess model performance across training, testing, and all data.
    """
    # Generate Predictions
    y_train_pred, y_train_prob = generate_predictions(model, X_train)
    y_test_pred, y_test_prob = generate_predictions(model, X_test)
    all_data_pred, all_data_prob = generate_predictions(model, X_all)

    # Calculate Metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_prob)
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_prob)
    all_data_metrics = calculate_metrics(Y_all, all_data_pred, all_data_prob)

    # Print Metrics
    if print_metrics:
        for metric_name in train_metrics:
            print(
                f"{metric_name}\nTrain: {train_metrics[metric_name]:.5f}  |  "
                f"Test: {test_metrics[metric_name]:.5f}  |  "
                f"All Data: {all_data_metrics[metric_name]:.5f}\n",
            )

    # Plot ROC Curve if requested
    if plot:
        plt.figure(figsize=(10, 7))
        plot_roc_curve(y_train, y_train_prob, "Train")
        plot_roc_curve(y_test, y_test_prob, "Test")
        # plot_roc_curve(Y_all, all_data_prob, "All Data")
        plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("Receiver Operating Characteristic (ROC) Curve")
        plt.legend()
        plt.show()

    if return_test_metrics:
        return test_metrics

In [ ]:
y_test[:, 15].sum()

In [ ]:
i = 15

for weighting_scheme in models:
    print(f"\n{weighting_scheme}")
    print(
        assess_model_performance(
            models[weighting_scheme][i],
            X_train,
            y_train[:, i],
            X_test,
            y_test[:, i],
            X,
            Y.toarray()[:, i],
        ),
    )

In [ ]:
# Calculate class weights to scale positive weights
total_negative = len(y_train) - sum(y_train)
total_positive = sum(y_train)

scale_pos_weight = total_negative / (
    total_positive // 1
)  # 1.5 for two_weeks and 4 for one_month | use sqrt if extremely imbalanced

# Single XGBoost Classifier
xgb_model = xgb.XGBClassifier(
    objective="binary:logistic",
    random_state=23,
    scale_pos_weight=scale_pos_weight,
)
xgb_model.fit(X_train, y_train)

In [ ]:
### ASSESS MODEL PERFORMANCE ###

# Predict labels for train, test, and all data
y_train_pred = xgb_model.predict(X_train)
y_train_prob = xgb_model.predict_proba(X_train)[:, 1]
y_test_pred = xgb_model.predict(X_test)
y_test_prob = xgb_model.predict_proba(X_test)[:, 1]

# Balanced Accuracy
y_train_accuracy = balanced_accuracy_score(y_train, y_train_pred)
y_test_accuracy = balanced_accuracy_score(y_test, y_test_pred)

# F1 Score
y_train_f1 = f1_score(y_train, y_train_pred)
y_test_f1 = f1_score(y_test, y_test_pred)

# Precision
y_train_precision = precision_score(y_train, y_train_pred)
y_test_precision = precision_score(y_test, y_test_pred)

# Recall
y_train_recall = recall_score(y_train, y_train_pred)
y_test_recall = recall_score(y_test, y_test_pred)

# AUROC
y_train_auroc = roc_auc_score(y_train, y_train_prob)
y_test_auroc = roc_auc_score(y_test, y_test_prob)

# AUC-PR (Area Under the Precision-Recall Curve)
y_train_p, y_train_r, _ = precision_recall_curve(y_train, y_train_prob)
y_test_p, y_test_r, _ = precision_recall_curve(y_test, y_test_prob)

y_train_auc_pr = auc(y_train_r, y_train_p)
y_test_auc_pr = auc(y_test_r, y_test_p)

# Average Precision Score (APS)
y_train_aps = average_precision_score(y_train, y_train_pred)
y_test_aps = average_precision_score(y_test, y_test_pred)

# Print Metrics
print(
    f"Balanced Accuracy\nTrain: {y_train_accuracy:.5f}  |  Test: {y_test_accuracy:.5f}\n",
)
print(
    f"F1 Score\nTrain: {y_train_f1:.5f}  |  Test: {y_test_f1:.5f}\n",
)
print(
    f"Precision\nTrain: {y_train_precision:.5f}  |  Test: {y_test_precision:.5f}\n",
)
print(
    f"Recall\nTrain: {y_train_recall:.5f}  |  Test: {y_test_recall:.5f}\n",
)
print(
    f"AUROC\nTrain: {y_train_auroc:.5f}  |  Test: {y_test_auroc:.5f}\n",
)
print(
    f"AUC-PR\nTrain: {y_train_auc_pr:.5f}  |  Test: {y_test_auc_pr:.5f}\n",
)
print(
    f"Average Precision Score\nTrain: {y_train_aps:.5f}  |  Test: {y_test_aps:.5f}\n",
)

# Plot ROC Curve
fpr_train, tpr_train, _ = roc_curve(y_train, y_train_prob)
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_prob)

# Plot Information
plt.figure(figsize=(10, 7))
plt.plot(fpr_train, tpr_train, label=f"Train AUROC={y_train_auroc:.3f}")
plt.plot(fpr_test, tpr_test, label=f"Test AUROC={y_test_auroc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend()
plt.show()

In [ ]:
xgb.plot_importance(xgb_model, max_num_features=20)
plt.show()

In [ ]:
# CAREFUL, one needs to match feature ids to actual FHIR features before interpretation
# Assess which features are the most important

# Get feature importances
feature_importances = xgb_model.feature_importances_

# Create a list of tuples (feature, importance) and sort it by importance in descending order
sorted_importances = sorted(
    zip(selected_features, feature_importances),
    key=lambda x: x[1],
    reverse=True,
)

# Display the top 10 most important features
top_features = sorted_importances[:10]
for feature, importance in top_features:
    print(f"{feature}: {importance}")

In [ ]:
# Save current predictions, labels, and probabilities to disk
np.save(f"{DATA_ROOT}/xgboost_y_test_pred_{TASK}.npy", y_test_pred)
np.save(f"{DATA_ROOT}/xgboost_y_test_pred_{TASK}_labels.npy", y_test)
np.save(
    f"{DATA_ROOT}/xgboost_y_test_pred_{TASK}_prob.npy",
    xgb_model.predict_proba(X_test),
)

In [ ]:
### SCRIPT FOR K-FOLD VALIDATION ###
N_FOLDS = 10

# Initialize StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=23)

# Initialize lists to store performance metrics for each fold
accuracy_scores = []
f1_scores = []
precisions = []
recalls = []
aurocs = []
auc_prs = []
average_precision_scores = []

# Perform k-fold cross-validation
for train_index, test_index in tqdm(
    stratified_kfold.split(X, Y),
    desc=f"{N_FOLDS}-Fold Validation",
    unit=" Model(s)",
):
    # Get the relevant train and test data
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = Y[train_index], Y[test_index]

    # Create a new XGBoost model for each fold
    xgb_model = xgb.XGBClassifier(
        objective="binary:logistic",
        random_state=23,
        scale_pos_weight=scale_pos_weight,
    )

    # Train the model on the training fold
    xgb_model.fit(X_train_fold, y_train_fold)

    # Predict on the test fold
    y_pred_fold = xgb_model.predict(X_test_fold)

    # Calculate performance metrics
    accuracy_fold = balanced_accuracy_score(y_test_fold, y_pred_fold)
    f1_fold = f1_score(y_test_fold, y_pred_fold)
    precision_fold = precision_score(y_test_fold, y_pred_fold)
    recall_fold = recall_score(y_test_fold, y_pred_fold)
    auroc_fold = roc_auc_score(y_test_fold, y_pred_fold)
    p_fold, r_fold, _ = precision_recall_curve(y_test_fold, y_pred_fold)
    auc_pr_fold = auc(r_fold, p_fold)
    average_precision_score_fold = average_precision_score(y_test_fold, y_pred_fold)

    # Append metrics to lists
    accuracy_scores.append(accuracy_fold)
    f1_scores.append(f1_fold)
    precisions.append(precision_fold)
    recalls.append(recall_fold)
    aurocs.append(auroc_fold)
    auc_prs.append(auc_pr_fold)
    average_precision_scores.append(average_precision_score_fold)

# Print average metrics across all folds
print(f"Average Balanced Accuracy: {sum(accuracy_scores) / N_FOLDS:.5f}")
print(f"Average F1 Score: {sum(f1_scores) / N_FOLDS:.5f}")
print(f"Average Precision: {sum(precisions) / N_FOLDS:.5f}")
print(f"Average Recall: {sum(recalls) / N_FOLDS:.5f}")
print(f"Average AUROC: {sum(aurocs) / N_FOLDS:.5f}")
print(f"Average AUC-PR: {sum(auc_prs) / N_FOLDS:.5f}")
print(f"Average Precision Score: {sum(average_precision_scores) / N_FOLDS:.5f}")

In [ ]:
# # Generate labels for the dataset if necessary
# labes_for_given_token = []; given_token = '8135'; de = []
#
# for idx, patient in tqdm(dataset.iterrows(), desc="Loading Tokens", unit=" Patients"):
#     if given_token in list(patient['event_tokens_2048'].split(' ')):
#         # print(patient); break
#         de.append(patient['label'])
#
# print(f"Total Occurrences: {len(labes_for_given_token)}, Positive Labels: {sum(labes_for_given_token)}")